In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!ls "/content/drive"
!pip install ijson

In [ ]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

!which python
!/usr/local/bin/python -V

import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
# os.environ['PYSPARK_PYTHON'] = "/usr/bin/python"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")


In [ ]:
################## spark declare #######################
from pyspark import SparkContext
from pyspark import SparkConf

#sc.stop()
sc = SparkContext()

In [7]:
import ijson
import re
import os, sys

f = open("/content/drive/My Drive/jsonJieba-tran/jsonJieba-tran.json", mode = 'r', encoding='UTF-8')
objects = ijson.items(f, 'item')
num = 0

####################### Handle json ###############################
# Extract noun. plus wiki_index -> Word:Wiki_Index

for p, article in enumerate(list(objects)):
    # print(article["id"], \
    # article["title"], \
    # article["content"])

    if (p % 1000 == 0):
        num = int(p / 1000)
        os.makedirs("./test/TEST" + str(num) + "/")

    with open("./test/TEST" + str(num) + "/" + str(article["id"]) + ".txt", mode = 'w', encoding='UTF-8') as f:
        w_list = article["content"].split("/")
        id_ = str(article["id"])
        word = []

        for m, i in enumerate(w_list):
          if (i != ""):
            type_ = i.split()

            for j, k in enumerate(type_):                
              if (j > 0 and k.find("n") >= 0):
                  word.append(type_[0] + ":" + id_)

        str1 = " ".join(word)
        f.write(str1)

In [ ]:
####################### Merge word #############################
# Use spark to merge the same word and different wiki_index
# e.g. word wiki_index:wiki_count(apple 113333:2 333333:4)

def test_(x):

  temp = str(x[0])
  name, id = temp.split(":")
  freq = id + ":" + str(x[1])

  return (name, freq)

filepath = './In_Index'

if (not os.path.exists(filepath)):
  os.mkdir(filepath)


for a in (os.listdir("./test")):

  data_file = "./test/" + a + "/"
  text_file = sc.textFile(data_file)

  counts = text_file.flatMap(lambda line: line.split()) \
              .map(lambda word: (word, 1)) \
              .reduceByKey(lambda a, b: a + b) \
              .map(test_) \
              .reduceByKey(lambda a, b: a + " " + b) \
              .sortBy(lambda x: x[0], ascending=True) \
              .collect()

  with open("./In_Index/" + a + ".txt", mode='w', encoding="UTF-8") as f:   
    for i in counts:
      f.write(i[0] + "\t" + i[1] + "\n")

In [15]:
################## Merge parts of In_Index ########################
# Merge part to become one Inverted_Index
# Use Inverted_Index to answer the ques

data_file = "./In_Index"

text_file = sc.textFile(data_file)
wordcountsRDD = text_file.map(lambda line: line.split("\t", 1)) \
             .reduceByKey(lambda a, b: a + " " + b) \
             .sortBy(lambda x: x[0], ascending = False) \
             .collect()

with open("./Inverted_Index", mode='w', encoding="UTF-8") as f:
  for i in wordcountsRDD:    
    f.write(i[0] + "\t" + i[1] + "\n")